# Capstone Project on the Battle of Neighborhoods: Weeks 1&2 Combined:
# Understanding the Problem with the situation and solving it optimally:

# Background
According to Bloomberg News, the London Housing Market is in a rut. It is now facing a number of different headwinds, including the prospect of higher taxes and a warning from the Bank of England that U.K. home values could fall as much as 30 percent in the event of a disorderly exit from the European Union. More specifically, four overlooked cracks suggest that the London market may be in worse shape than many realize: hidden price falls, record-low sales, homebuilder exodus and tax hikes addressing overseas buyers of homes in England and Wales.

# Business Problem
In this scenario, it is urgent to adopt machine learning tools in order to assist homebuyers clientele in London to make wise and effective decisions. As a result, the business problem we are currently posing is: how could we provide support to homebuyers clientele in to purchase a suitable real estate in London in this uncertain economic and financial scenario?

To solve this business problem, we are going to cluster London neighborhoods in order to recommend venues and the current average price of real estate where homebuyers can make a real estate investment. We will recommend profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.

* Data section
Data on London properties and the relative price paid data were extracted from the HM Land Registry (http://landregistry.data.gov.uk/). The following fields comprise the address data included in Price Paid Data: Postcode; PAON Primary Addressable Object Name. Typically the house number or name; SAON Secondary Addressable Object Name. If there is a sub-building, for example, the building is divided into flats, there will be a SAON; Street; Locality; Town/City; District; County.

---->In order to explore and target recommended locations across different venues according to the presence of amenities and essential facilities, we will access data through FourSquare API interface and arrange them as a dataframe for visualization. By merging data on London properties and the relative price paid data from the HM Land Registry and data on amenities and essential facilities surrounding such properties from FourSquare API interface, we will be able to recommend profitable real estate investments.

* Methodology section
The Methodology section will describe the main components of our analysis and predication system. The Methodology section comprises four stages:

1. Collect Inspection Data
2. Explore and Understand Data
3. Data preparation and preprocessing 
4. Modeling

In [1]:
import os # Operating System
import numpy as np
import pandas as pd
import datetime as dt # Datetime
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium #import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0           conda-forge
    geopy:          

In [2]:
#Read the data for examination (Source: http://landregistry.data.gov.uk/)
df_ppd = pd.read_csv("http://prod2.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2018.csv")

* Before Using the data, we have to explore and undertsnad it.

# Exploring and Understandining the data:
 
 * We read the dataset that we collected from the HM Land Registry website into a pandas' data frame and display the first five rows of it as follows:

In [3]:
df_ppd.head(5)

,{79A74E21-D11E-1289-E053-6B04A8C01627},770000,2018-09-25 00:00,SK7 1AR,D,N,F,5,Unnamed: 8,OAK MEADOW,BRAMHALL,STOCKPORT,STOCKPORT.1,GREATER MANCHESTER,A,A.1
0,{79A74E21-D11F-1289-E053-6B04A8C01627},253500,2018-09-24 00:00,M6 8GQ,D,N,F,1,NaN,RIVINGTON ROAD,NaN,SALFORD,SALFORD,GREATER MANCHESTER,A,A
1,{79A74E21-D120-1289-E053-6B04A8C01627},231950,2018-09-28 00:00,WA3 2UE,D,Y,F,35,NaN,STONEACRE CLOSE,LOWTON,WARRINGTON,WIGAN,GREATER MANCHESTER,A,A
2,{79A74E21-D121-1289-E053-6B04A8C01627},112500,2018-08-29 00:00,OL6 6RJ,S,N,F,102,NaN,THORNFIELD GROVE,NaN,ASHTON-UNDER-LYNE,TAMESIDE,GREATER MANCHESTER,A,A
3,{79A74E21-D122-1289-E053-6B04A8C01627},184995,2018-06-15 00:00,M46 0TW,S,Y,F,37,NaN,THREADNEEDLE PLACE,ATHERTON,MANCHESTER,WIGAN,GREATER MANCHESTER,A,A
4,{79A74E21-D123-1289-E053-6B04A8C01627},214995,2018-09-28 00:00,M28 3XS,D,Y,L,9,NaN,MARPLE GARDENS,WORSLEY,MANCHESTER,SALFORD,GREATER MANCHESTER,A,A


In [7]:
df_ppd.shape()

TypeError: 'tuple' object is not callable

Our dataset consists of over 700000 rows and 16 columns. We will now prepare and preprocess data accordingly.

# 3. Data preparation and preprocessing:
At this stage, we prepare our dataset for the modeling process, opting for the most suitable machine learning algorithm for our scope. Accordingly, we perform the following steps:

* Rename the column names
* Format the date column
* Sort data by date of sale
* Select data only for the city of London
* Make a list of street names in London
* Calculate the street-wise average price of the property
* Read the street-wise coordinates into a data frame, eliminating recurring word London from individual names
* Join the data to find the coordinates of locations which fit into client's budget
* Plot recommended locations on London map along with current market prices

In [5]:

# Assign meaningful column names
df_ppd.columns = ['TUID', 'Price', 'Date_Transfer', 'Postcode', 'Prop_Type', 'Old_New', 'Duration', 'PAON', \
                  'SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 'PPD_Cat_Type', 'Record_Status']
# Format the date column
df_ppd['Date_Transfer'] = df_ppd['Date_Transfer'].apply(pd.to_datetime)

# Delete all obsolete transactions which were done before 2016
df_ppd.drop(df_ppd[df_ppd.Date_Transfer.dt.year < 2016].index, inplace=True)

# Sort by Date of Sale
df_ppd.sort_values(by=['Date_Transfer'],ascending=[False],inplace=True)
df_ppd_london = df_ppd.query("Town_City == 'LONDON'")

# Make a list of street names in LONDON
streets = df_ppd_london['Street'].unique().tolist()
df_grp_price = df_ppd_london.groupby(['Street'])['Price'].mean().reset_index()

# Give meaningful names to the columns
df_grp_price.columns = ['Street', 'Avg_Price']

#Input your Budget's Upper Limit and Lower Limit - Find the locations df_grp_price which fits your budget
df_affordable = df_grp_price.query("(Avg_Price >= 2200000) & (Avg_Price <= 2500000)")
# Display the dataframe
df_affordable

,Street,Avg_Price
196,ALBION SQUARE,2.450000e+06
390,ANHALT ROAD,2.435000e+06
405,ANSDELL TERRACE,2.250000e+06
422,APPLEGARTH ROAD,2.400000e+06
855,BARONSMEAD ROAD,2.375000e+06
981,BEAUCLERC ROAD,2.480000e+06
1102,BELVEDERE DRIVE,2.340000e+06
1215,BICKENHALL STREET,2.208500e+06
1253,BIRCHLANDS AVENUE,2.217000e+06
1553,BRAMPTON GROVE,2.456875e+06


In [6]:
import numpy as np
import datetime as DT
import hmac
from geopy.geocoders import Nominatim
from geopy.distance import vincenty
# import k-means from clustering stage
from sklearn.cluster import KMeans
for index, item in df_affordable.iterrows():
    print(f"index: {index}")
    print(f"item: {item}")
    print(f"item.Street only: {item.Street}")

index: 196
item: Street       ALBION SQUARE
Avg_Price         2.45e+06
Name: 196, dtype: object
item.Street only: ALBION SQUARE
index: 390
item: Street       ANHALT ROAD
Avg_Price      2.435e+06
Name: 390, dtype: object
item.Street only: ANHALT ROAD
index: 405
item: Street       ANSDELL TERRACE
Avg_Price           2.25e+06
Name: 405, dtype: object
item.Street only: ANSDELL TERRACE
index: 422
item: Street       APPLEGARTH ROAD
Avg_Price            2.4e+06
Name: 422, dtype: object
item.Street only: APPLEGARTH ROAD
index: 855
item: Street       BARONSMEAD ROAD
Avg_Price          2.375e+06
Name: 855, dtype: object
item.Street only: BARONSMEAD ROAD
index: 981
item: Street       BEAUCLERC ROAD
Avg_Price          2.48e+06
Name: 981, dtype: object
item.Street only: BEAUCLERC ROAD
index: 1102
item: Street       BELVEDERE DRIVE
Avg_Price           2.34e+06
Name: 1102, dtype: object
item.Street only: BELVEDERE DRIVE
index: 1215
item: Street       BICKENHALL STREET
Avg_Price           2.2085e+06
N

In [10]:
geolocator = Nominatim()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.22.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  if __name__ == '__main__':


In [12]:
df_affordable['city_coord'] = df_affordable['Street'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [16]:
df_affordable

,Street,Avg_Price,city_coord,Latitude,Longitude
196,ALBION SQUARE,2.450000e+06,"(-41.27375755, 173.28939323910353)",-41.273758,173.289393
390,ANHALT ROAD,2.435000e+06,"(51.4803164, -0.1668011)",51.480316,-0.166801
405,ANSDELL TERRACE,2.250000e+06,"(51.4998899, -0.1891027)",51.499890,-0.189103
422,APPLEGARTH ROAD,2.400000e+06,"(53.7486539, -0.3266704)",53.748654,-0.326670
855,BARONSMEAD ROAD,2.375000e+06,"(51.4773147, -0.239457)",51.477315,-0.239457
981,BEAUCLERC ROAD,2.480000e+06,"(30.2114523, -81.6179807)",30.211452,-81.617981
1102,BELVEDERE DRIVE,2.340000e+06,"(38.0728178, -78.4587964)",38.072818,-78.458796
1215,BICKENHALL STREET,2.208500e+06,"(51.5212014, -0.1589082)",51.521201,-0.158908
1253,BIRCHLANDS AVENUE,2.217000e+06,"(51.4483941, -0.1604676)",51.448394,-0.160468
1553,BRAMPTON GROVE,2.456875e+06,"(51.5899607, -0.3185249)",51.589961,-0.318525


In [14]:
df_affordable[['Latitude', 'Longitude']] = df_affordable['city_coord'].apply(pd.Series)

In [15]:
df_affordable

,Street,Avg_Price,city_coord,Latitude,Longitude
196,ALBION SQUARE,2.450000e+06,"(-41.27375755, 173.28939323910353)",-41.273758,173.289393
390,ANHALT ROAD,2.435000e+06,"(51.4803164, -0.1668011)",51.480316,-0.166801
405,ANSDELL TERRACE,2.250000e+06,"(51.4998899, -0.1891027)",51.499890,-0.189103
422,APPLEGARTH ROAD,2.400000e+06,"(53.7486539, -0.3266704)",53.748654,-0.326670
855,BARONSMEAD ROAD,2.375000e+06,"(51.4773147, -0.239457)",51.477315,-0.239457
981,BEAUCLERC ROAD,2.480000e+06,"(30.2114523, -81.6179807)",30.211452,-81.617981
1102,BELVEDERE DRIVE,2.340000e+06,"(38.0728178, -78.4587964)",38.072818,-78.458796
1215,BICKENHALL STREET,2.208500e+06,"(51.5212014, -0.1589082)",51.521201,-0.158908
1253,BIRCHLANDS AVENUE,2.217000e+06,"(51.4483941, -0.1604676)",51.448394,-0.160468
1553,BRAMPTON GROVE,2.456875e+06,"(51.5899607, -0.3185249)",51.589961,-0.318525


In [ ]:
df = df_affordable.drop(columns=['city_coord'])

In [19]:
df

,Street,Avg_Price,Latitude,Longitude
196,ALBION SQUARE,2.450000e+06,-41.273758,173.289393
390,ANHALT ROAD,2.435000e+06,51.480316,-0.166801
405,ANSDELL TERRACE,2.250000e+06,51.499890,-0.189103
422,APPLEGARTH ROAD,2.400000e+06,53.748654,-0.326670
855,BARONSMEAD ROAD,2.375000e+06,51.477315,-0.239457
981,BEAUCLERC ROAD,2.480000e+06,30.211452,-81.617981
1102,BELVEDERE DRIVE,2.340000e+06,38.072818,-78.458796
1215,BICKENHALL STREET,2.208500e+06,51.521201,-0.158908
1253,BIRCHLANDS AVENUE,2.217000e+06,51.448394,-0.160468
1553,BRAMPTON GROVE,2.456875e+06,51.589961,-0.318525


In [21]:
address = 'London, UK'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London City are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.22.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of London City are 51.5073219, -0.1276474.


In [22]:

# create map of London using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, price, street in zip(df['Latitude'], df['Longitude'], df['Avg_Price'], df['Street']):
    label = '{}, {}'.format(street, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

In [28]:
#Define Foursquare Credentials and Version
CLIENT_ID = 'KI3TR0QO4JOKMFELOMF3WSOOI3HFNBF5YLW354MYWBKDHEX3' # Foursquare ID
CLIENT_SECRET = 'QF4ZBLJRBV4BQX52DVWUPEHJ14A2UJABPCZARZQZYTKIISUD' # Foursquare Secret
VERSION = '20181206' # Foursquare API

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KI3TR0QO4JOKMFELOMF3WSOOI3HFNBF5YLW354MYWBKDHEX3
CLIENT_SECRET:QF4ZBLJRBV4BQX52DVWUPEHJ14A2UJABPCZARZQZYTKIISUD


* We can now proceed to the Modeling phase. We will analyze neighborhoods to recommend real estates where home buyers can make a real estate investment. We will then recommend profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.

# Week 2 of 2: Post Undestanding the problem and deploying a solution for the problem
# Modelling
After exploring the dataset and gaining insights into it, we are ready to use the clustering methodology to analyze real estates. We will use the k-means clustering technique as it is fast and efficient in terms of computational cost, is highly flexible to account for mutations in real estate market in London and is accurate.

In [36]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [37]:

# Run the above function on each location and create a new dataframe called location_venues and display it.
location_venues = getNearbyVenues(names=df['Street'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                 )
 

ALBION SQUARE
ANHALT ROAD
ANSDELL TERRACE
APPLEGARTH ROAD
BARONSMEAD ROAD
BEAUCLERC ROAD
BELVEDERE DRIVE
BICKENHALL STREET
BIRCHLANDS AVENUE
BRAMPTON GROVE
BRIARDALE GARDENS
BROOKWAY
BURBAGE ROAD
BURY WALK
CALLCOTT STREET
CAMPDEN HILL ROAD
CAMPION ROAD
CANNING PLACE
CARLISLE ROAD
CARLTON GARDENS
CARLYLE COURT
CHALCOT SQUARE
CHARLES LANE
CHELSEA CRESCENT
CHESTER CLOSE NORTH
CHEYNE COURT
CHEYNE ROW
CHISWICK MALL
CITY ROAD
CLARENDON STREET
CLONCURRY STREET
COLBECK MEWS
COLLEGE CRESCENT
CORNWALL TERRACE MEWS
COURT LANE GARDENS
CRESCENT GROVE
DALEBURY ROAD
DEWHURST ROAD
DORIA ROAD
DOWNSHIRE HILL
DUCHESS WALK
ECCLESTON SQUARE MEWS
EGBERT STREET
EGERTON PLACE
ELM PARK ROAD
FLORAL STREET
FRANK DIXON WAY
FULTON MEWS
GERARD ROAD
GERRARD ROAD
GIRDLERS ROAD
GLOUCESTER CRESCENT
GORDON PLACE
GRAFTON SQUARE
GRAHAM TERRACE
HARMAN DRIVE
HARRIS STREET
HAVANNAH STREET
HAZLEWELL ROAD
HEREFORD MEWS
HERONDALE AVENUE
HIGHGATE HIGH STREET
HIGHWOOD HILL
HILLGATE PLACE
HOLLYCROFT AVENUE
HOLLYWOOD MEWS
HONEYWELL

In [38]:
location_venues

,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ALBION SQUARE,-41.273758,173.289393,The Free House,-41.273340,173.287364,Bar
1,ALBION SQUARE,-41.273758,173.289393,The Indian Cafe,-41.273308,173.286530,Indian Restaurant
2,ALBION SQUARE,-41.273758,173.289393,Queen's Gardens,-41.273671,173.291383,Park
3,ALBION SQUARE,-41.273758,173.289393,Urban,-41.274355,173.286317,New American Restaurant
4,ALBION SQUARE,-41.273758,173.289393,Fish Stop,-41.276010,173.289592,Fish & Chips Shop
5,ALBION SQUARE,-41.273758,173.289393,Deville Cafe,-41.271941,173.285535,Beer Garden
6,ALBION SQUARE,-41.273758,173.289393,Fresh Choice,-41.272194,173.287218,Supermarket
7,ALBION SQUARE,-41.273758,173.289393,The Bridge Street Collective,-41.272520,173.285517,Café
8,ALBION SQUARE,-41.273758,173.289393,Mango,-41.274460,173.285345,Indian Restaurant
9,ALBION SQUARE,-41.273758,173.289393,Hopgood's,-41.274749,173.283831,Restaurant


In [40]:
location_venues.groupby('Street').count()

,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Street,,,,,,
ALBION SQUARE,27,27,27,27,27,27
ANHALT ROAD,17,17,17,17,17,17
ANSDELL TERRACE,43,43,43,43,43,43
APPLEGARTH ROAD,5,5,5,5,5,5
BARONSMEAD ROAD,13,13,13,13,13,13
BEAUCLERC ROAD,4,4,4,4,4,4
BELVEDERE DRIVE,3,3,3,3,3,3
BICKENHALL STREET,58,58,58,58,58,58
BIRCHLANDS AVENUE,10,10,10,10,10,10


In [41]:
# unique categories list
print('There are {} uniques categories.'.format(len(location_venues['Venue Category'].unique())))

There are 341 uniques categories.


In [42]:
location_venues.shape

(4355, 7)

In [43]:
# one hot encoding
venues_onehot = pd.get_dummies(location_venues[['Venue Category']], prefix="", prefix_sep="")

# add street column back to dataframe
venues_onehot['Street'] = location_venues['Street'] 

# move street column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])

#fixed_columns
venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head()

,Street,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Apres Ski Bar,Arcade,Argentinian Restaurant,...,Volleyball Court,Warehouse Store,Waterfront,Weight Loss Center,Windmill,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
london_grouped = venues_onehot.groupby('Street').mean().reset_index()
london_grouped

,Street,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Apres Ski Bar,Arcade,Argentinian Restaurant,...,Volleyball Court,Warehouse Store,Waterfront,Weight Loss Center,Windmill,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,ALBION SQUARE,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.0,0.00,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
1,ANHALT ROAD,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.0,0.00,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
2,ANSDELL TERRACE,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.0,0.00,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
3,APPLEGARTH ROAD,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.0,0.00,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
4,BARONSMEAD ROAD,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.0,0.00,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
5,BEAUCLERC ROAD,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.0,0.00,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
6,BELVEDERE DRIVE,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.0,0.00,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
7,BICKENHALL STREET,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.0,0.00,0.000000,0.0,0.0,0.017241,0.000000,0.017241,0.017241,0.000000
8,BIRCHLANDS AVENUE,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.0,0.00,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
9,BRAMPTON GROVE,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.0,0.00,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000


In [45]:
london_grouped.shape

(149, 342)

In [46]:
# top 5 facilities with regards to area for best real estate investments:

num_top_venues = 5

for hood in london_grouped['Street']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['Street'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ALBION SQUARE----
               venue  freq
0               Café  0.22
1  Indian Restaurant  0.07
2                Bar  0.07
3         Restaurant  0.07
4        Coffee Shop  0.07


----ANHALT ROAD----
                venue  freq
0                 Pub  0.24
1       Grocery Store  0.12
2   French Restaurant  0.12
3  English Restaurant  0.06
4         Art Gallery  0.06


----ANSDELL TERRACE----
                venue  freq
0          Restaurant  0.09
1  Italian Restaurant  0.07
2               Hotel  0.07
3      Clothing Store  0.07
4           Juice Bar  0.07


----APPLEGARTH ROAD----
            venue  freq
0          Casino   0.2
1  Sandwich Place   0.2
2       Nightclub   0.2
3             Bar   0.2
4             Pub   0.2


----BARONSMEAD ROAD----
                 venue  freq
0    Food & Drink Shop  0.15
1       Breakfast Spot  0.08
2                 Park  0.08
3  Indie Movie Theater  0.08
4                  Pub  0.08


----BEAUCLERC ROAD----
               venue  freq
0         

                        venue  freq
0  Construction & Landscaping  0.33
1              Cosmetics Shop  0.33
2               Deli / Bodega  0.33
3     New American Restaurant  0.00
4        Outdoor Supply Store  0.00


----FRANK DIXON WAY----
                  venue  freq
0  Gym / Fitness Center  0.17
1                  Park  0.17
2            Playground  0.17
3                  Farm  0.17
4                  Café  0.17


----FULTON MEWS----
                venue  freq
0               Hotel  0.16
1                 Pub  0.08
2         Coffee Shop  0.07
3                Café  0.06
4  Chinese Restaurant  0.06


----GERARD ROAD----
                  venue  freq
0      Business Service  0.33
1     Fish & Chips Shop  0.33
2     Convenience Store  0.33
3             Nightclub  0.00
4  Outdoor Supply Store  0.00


----GERRARD ROAD----
                           venue  freq
0                            Pub   0.5
1           Fast Food Restaurant   0.5
2                           Park   0.0
3  Pape

----MUNDEN STREET----
               venue  freq
0              Hotel  0.10
1                Pub  0.10
2           Platform  0.08
3               Café  0.08
4  Indian Restaurant  0.06


----NORTH CIRCULAR ROAD----
                venue  freq
0      Cosmetics Shop  0.25
1         Flower Shop  0.25
2  Chinese Restaurant  0.25
3  Miscellaneous Shop  0.25
4   Accessories Store  0.00


----NOTTINGHAM STREET----
                 venue  freq
0               Bakery  0.25
1            Wine Shop  0.25
2                Trail  0.25
3  Sporting Goods Shop  0.25
4    Polish Restaurant  0.00


----OAKLEY STREET----
                     venue  freq
0     Gym / Fitness Center  0.33
1         Volleyball Court  0.33
2                   Lounge  0.33
3  New American Restaurant  0.00
4     Outdoor Supply Store  0.00


----OAKWOOD COURT----
         venue  freq
0         Food   0.2
1  Golf Course   0.2
2  Pizza Place   0.2
3   Smoke Shop   0.2
4  Video Store   0.2


----OBSERVATORY GARDENS----
            ve

                venue  freq
0               Hotel  0.09
1          Restaurant  0.09
2           Juice Bar  0.07
3  Italian Restaurant  0.07
4      Clothing Store  0.07


----SOUTHWOOD LAWN ROAD----
                venue  freq
0                 Pub  0.30
1   Indian Restaurant  0.06
2              Bakery  0.06
3                Café  0.06
4  Italian Restaurant  0.06


----SOVEREIGN PARK----
                  venue  freq
0                   Gym  0.33
1    Chinese Restaurant  0.33
2           Pizza Place  0.33
3  Outdoor Supply Store  0.00
4      Pedestrian Plaza  0.00


----ST OSWALDS PLACE----
                venue  freq
0                Café  0.14
1                 Pub  0.11
2  Italian Restaurant  0.07
3             Gay Bar  0.07
4      Cricket Ground  0.07


----ST PETERS SQUARE----
                venue  freq
0  Italian Restaurant  0.22
1      Ice Cream Shop  0.10
2                Café  0.09
3               Hotel  0.08
4      Sandwich Place  0.04


----STAFFORD TERRACE----
            

In [47]:
# defining a function to identify most common areas for real estate investments:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [48]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Street']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [49]:
#create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Street'] = london_grouped['Street']

for ind in np.arange(london_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

In [50]:
venues_sorted.head()

,Street,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ALBION SQUARE,Café,Indian Restaurant,Restaurant,Pub,Coffee Shop,Bar,Burger Joint,Fish & Chips Shop,French Restaurant,New American Restaurant
1,ANHALT ROAD,Pub,Grocery Store,French Restaurant,Farmers Market,Garden,Plaza,English Restaurant,Cocktail Bar,Japanese Restaurant,Art Gallery
2,ANSDELL TERRACE,Restaurant,Hotel,Italian Restaurant,Juice Bar,Clothing Store,Bakery,Garden,Pub,Indian Restaurant,Sushi Restaurant
3,APPLEGARTH ROAD,Casino,Bar,Sandwich Place,Pub,Nightclub,Factory,Egyptian Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant
4,BARONSMEAD ROAD,Food & Drink Shop,Movie Theater,Thai Restaurant,Park,Farmers Market,Nature Preserve,Coffee Shop,Restaurant,Pub,Breakfast Spot


In [52]:
venues_sorted.shape

(149, 11)

In [53]:
london_grouped.shape

(149, 342)

In [55]:
london_grouped=df

* Therefore,after conducting our inspection of venues/facilities/amenities nearby the most profitable real estate investments in London, we could begin by clustering properties by venues/facilities/amenities nearby.

In [56]:
# distribution into 5 clusters:
# set number of clusters
kclusters = 5

london_grouped_clustering = london_grouped.drop('Street', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50]


array([1, 3, 0, 3, 2, 1, 2, 0, 0, 1, 3, 3, 3, 1, 2, 2, 1, 3, 0, 1, 4, 4,
       3, 1, 1, 0, 3, 4, 1, 0, 3, 2, 3, 2, 2, 4, 3, 3, 2, 0, 1, 2, 4, 0,
       4, 0, 0, 4, 0, 0], dtype=int32)

In [57]:
# dataframe into include clusters:

london_grouped_clustering=df
london_grouped_clustering.head()

,Street,Avg_Price,Latitude,Longitude
196,ALBION SQUARE,2450000.0,-41.273758,173.289393
390,ANHALT ROAD,2435000.0,51.480316,-0.166801
405,ANSDELL TERRACE,2250000.0,51.499890,-0.189103
422,APPLEGARTH ROAD,2400000.0,53.748654,-0.326670
855,BARONSMEAD ROAD,2375000.0,51.477315,-0.239457


In [58]:
london_grouped_clustering.shape

(162, 4)

In [59]:
df.shape

(162, 4)

In [60]:
london_grouped_clustering.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [61]:
df.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [62]:
# add clustering labels
london_grouped_clustering['Cluster Labels'] = kmeans.labels_

# merge london_grouped with london_data to add latitude/longitude for each neighborhood
london_grouped_clustering = london_grouped_clustering.join(venues_sorted.set_index('Street'), on='Street')

london_grouped_clustering.head(30) # check the last columns!

,Street,Avg_Price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
196,ALBION SQUARE,2.450000e+06,-41.273758,173.289393,1,Café,Indian Restaurant,Restaurant,Pub,Coffee Shop,Bar,Burger Joint,Fish & Chips Shop,French Restaurant,New American Restaurant
390,ANHALT ROAD,2.435000e+06,51.480316,-0.166801,3,Pub,Grocery Store,French Restaurant,Farmers Market,Garden,Plaza,English Restaurant,Cocktail Bar,Japanese Restaurant,Art Gallery
405,ANSDELL TERRACE,2.250000e+06,51.499890,-0.189103,0,Restaurant,Hotel,Italian Restaurant,Juice Bar,Clothing Store,Bakery,Garden,Pub,Indian Restaurant,Sushi Restaurant
422,APPLEGARTH ROAD,2.400000e+06,53.748654,-0.326670,3,Casino,Bar,Sandwich Place,Pub,Nightclub,Factory,Egyptian Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant
855,BARONSMEAD ROAD,2.375000e+06,51.477315,-0.239457,2,Food & Drink Shop,Movie Theater,Thai Restaurant,Park,Farmers Market,Nature Preserve,Coffee Shop,Restaurant,Pub,Breakfast Spot
981,BEAUCLERC ROAD,2.480000e+06,30.211452,-81.617981,1,Speakeasy,Automotive Shop,Pizza Place,Harbor / Marina,Farm,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit
1102,BELVEDERE DRIVE,2.340000e+06,38.072818,-78.458796,2,Pool,Playground,Athletics & Sports,Zoo,Falafel Restaurant,Egyptian Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space
1215,BICKENHALL STREET,2.208500e+06,51.521201,-0.158908,0,Pizza Place,Café,Hotel,Restaurant,Gastropub,Movie Theater,Garden,Bar,Italian Restaurant,Bakery
1253,BIRCHLANDS AVENUE,2.217000e+06,51.448394,-0.160468,0,Pub,Breakfast Spot,Brewery,Bakery,French Restaurant,Coffee Shop,Lake,Train Station,Chinese Restaurant,Falafel Restaurant
1553,BRAMPTON GROVE,2.456875e+06,51.589961,-0.318525,1,Construction & Landscaping,Home Service,Zoo,Farmers Market,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory


In [63]:
# Create Map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_grouped_clustering['Latitude'], london_grouped_clustering['Longitude'], london_grouped_clustering['Street'], london_grouped_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [64]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 0, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
405,2250000.0,Restaurant,Hotel,Italian Restaurant,Juice Bar,Clothing Store,Bakery,Garden,Pub,Indian Restaurant,Sushi Restaurant
1215,2208500.0,Pizza Place,Café,Hotel,Restaurant,Gastropub,Movie Theater,Garden,Bar,Italian Restaurant,Bakery
1253,2217000.0,Pub,Breakfast Spot,Brewery,Bakery,French Restaurant,Coffee Shop,Lake,Train Station,Chinese Restaurant,Falafel Restaurant
2225,2200000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2638,2250000.0,Hotel,Restaurant,Bookstore,Supermarket,Outdoor Sculpture,Outdoor Supply Store,Grocery Store,Gym / Fitness Center,Pharmacy,Bakery


In [65]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 1, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
196,2450000.0,Café,Indian Restaurant,Restaurant,Pub,Coffee Shop,Bar,Burger Joint,Fish & Chips Shop,French Restaurant,New American Restaurant
981,2480000.0,Speakeasy,Automotive Shop,Pizza Place,Harbor / Marina,Farm,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit
1553,2456875.0,Construction & Landscaping,Home Service,Zoo,Farmers Market,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory
1980,2492500.0,Supermarket,English Restaurant,Café,Hardware Store,Dry Cleaner,Gym,Rental Car Location,Park,Fast Food Restaurant,Pub
2136,2461000.0,Soccer Field,Spa,Sports Club,Windmill,Bus Station,Farm,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space


In [66]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 1, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
196,2450000.0,Café,Indian Restaurant,Restaurant,Pub,Coffee Shop,Bar,Burger Joint,Fish & Chips Shop,French Restaurant,New American Restaurant
981,2480000.0,Speakeasy,Automotive Shop,Pizza Place,Harbor / Marina,Farm,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit
1553,2456875.0,Construction & Landscaping,Home Service,Zoo,Farmers Market,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory
1980,2492500.0,Supermarket,English Restaurant,Café,Hardware Store,Dry Cleaner,Gym,Rental Car Location,Park,Fast Food Restaurant,Pub
2136,2461000.0,Soccer Field,Spa,Sports Club,Windmill,Bus Station,Farm,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space


In [67]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 3, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
390,2435000.0,Pub,Grocery Store,French Restaurant,Farmers Market,Garden,Plaza,English Restaurant,Cocktail Bar,Japanese Restaurant,Art Gallery
422,2400000.0,Casino,Bar,Sandwich Place,Pub,Nightclub,Factory,Egyptian Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant
1632,2397132.0,Grocery Store,Gym / Fitness Center,Coffee Shop,Italian Restaurant,Health & Beauty Service,Falafel Restaurant,Egyptian Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant
1797,2400000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1914,2445000.0,Construction & Landscaping,Dance Studio,Bar,Gym,Athletics & Sports,Grocery Store,Coworking Space,Farmers Market,English Restaurant,Ethiopian Restaurant


In [68]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 4, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2242,2.300000e+06,Farm,Zoo,Egyptian Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant
2406,2.286679e+06,Café,Pub,Coffee Shop,Bar,Italian Restaurant,Park,French Restaurant,Fish Market,Bakery,Speakeasy
2686,2.287500e+06,Pub,Brewery,Gift Shop,Gym / Fitness Center,Art Museum,Zoo,Falafel Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant
3377,2.298000e+06,Hotel,Zoo,Farm,Egyptian Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory
4285,2.265000e+06,Pub,Zoo,Farm,Egyptian Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory


# Results and Discussion section
* First of all, even though the London Housing Market may be in a rut, it is still an "ever-green" for business affairs.

* We may discuss our results under two main perspectives.

1) First, we may examine them according to neighborhoods/London areas. It is interesting to note that, although West London (Notting Hill, Kensington, Chelsea, Marylebone) and North-West London (Hampsted) might be considered highly profitable venues to purchase a real estate according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores, South-West London (Wandsworth, Balham) and North-West London (Isliington) are arising as next future elite venues with a wide range of amenities and facilities. Accordingly, one might target under-priced real estates in these areas of London in order to make a business affair.

2) In the second case, we analyzed the results according to the five clusters we have produced. Even though, all clusters could praise an optimal range of facilities and amenities, we have found two main patterns. The first pattern we are referring to, i.e. Clusters 0, 2 and 4, may target home buyers prone to live in 'green' areas with parks, waterfronts. Instead, the second pattern we are referring to, i.e. Clusters 1 and 3, may target individuals who love pubs, theatres and soccer.

# Conclusion of the project with Reccomendations for future:
* To sum up, according to Bloomberg News, the London Housing Market is in a rut. It is now facing a number of different headwinds, including the prospect of higher taxes and a warning from the Bank of England that U.K. home values could fall as much as 30 percent in the event of a disorderly exit from the European Union. In this scenario, it is urgent to adopt machine learning tools in order to assist homebuyers clientele in London to make wise and effective decisions. As a result, the business problem we were posing was: how could we provide support to homebuyers clientele in to purchase a suitable real estate in London in this uncertain economic and financial scenario?

To solve this business problem, we clustered London neighborhoods in order to recommend venues and the current average price of real estate where homebuyers can make a real estate investment. We recommended profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.

1) Therefore at first, we gathered data on London properties and the relative price paid data were extracted from the HM Land Registry (http://landregistry.data.gov.uk/). Moreover, to explore and target recommended locations across different venues according to the presence of amenities and essential facilities, we accessed data through FourSquare API interface and arranged them as a data frame for visualization. By merging data on London properties and the relative price paid data from the HM Land Registry and data on amenities and essential facilities surrounding such properties from FourSquare API interface, we were able to recommend profitable real estate investments.

2) Second, The Methodology section comprised four stages: 1. Collect Inspection Data; 2. Explore and Understand Data; 3. Data preparation and preprocessing; 4. Modeling. In particular, in the modeling section, we used the k-means clustering technique as it is fast and efficient in terms of computational cost, is highly flexible to account for mutations in real estate market in London and is accurate.

---->Finally, we drew the conclusion that even though the London Housing Market may be in a rut, it is still an "ever-green" for business affairs. We discussed our results under two main perspectives. First, we examined them according to neighborhoods/London areas. although West London (Notting Hill, Kensington, Chelsea, Marylebone) and North-West London (Hampsted) might be considered highly profitable venues to purchase a real estate according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores, South-West London (Wandsworth, Balham) and North-West London (Isliington) are arising as next future elite venues with a wide range of amenities and facilities. Accordingly, one might target under-priced real estates in these areas of London in order to make a business affair. Second, we analyzed our results according to the five clusters we produced. While Clusters 0, 2 and 4 may target home buyers prone to live in 'green' areas with parks, waterfronts, Clusters 1 and 3 may target individuals who love pubs, theatres and soccer.